# Task 2: Exploratory Data Analysis

## Objective
Analyze patterns and factors influencing financial inclusion in Ethiopia.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Load data
df_main = pd.read_excel('../data/processed/ethiopia_fi_unified_data_enriched.xlsx')
df_impact = pd.read_excel('../data/processed/impact_links_enriched.xlsx')

# Prepare data
df_main['observation_date'] = pd.to_datetime(df_main['observation_date'], errors='coerce')
observations = df_main[df_main['record_type'] == 'observation']
events = df_main[df_main['record_type'] == 'event']

print(f"Dataset loaded: {len(observations)} observations, {len(events)} events")

## 1. Dataset Overview

In [ ]:
# Dataset summary
print("=== DATASET OVERVIEW ===")
print(f"Record Types:")
print(df_main['record_type'].value_counts())
print(f"\nPillar Distribution:")
print(observations['pillar'].value_counts())
print(f"\nConfidence Levels:")
print(observations['confidence'].value_counts())

## 2. Access Analysis - Account Ownership

In [ ]:
# Account ownership trajectory
acc_data = observations[observations['indicator_code'] == 'ACC_OWNERSHIP'].sort_values('observation_date')

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=acc_data['observation_date'],
    y=acc_data['value_numeric'],
    mode='lines+markers',
    name='Account Ownership Rate',
    line=dict(width=3),
    marker=dict(size=8)
))

fig.update_layout(
    title="Ethiopia's Account Ownership Trajectory (2011-2024)",
    xaxis_title='Year',
    yaxis_title='Account Ownership Rate (%)',
    template='plotly_white'
)
fig.show()

# Growth rates
print("Growth Rates:")
for i in range(1, len(acc_data)):
    prev = acc_data.iloc[i-1]
    curr = acc_data.iloc[i]
    growth = ((curr['value_numeric'] - prev['value_numeric']) / prev['value_numeric']) * 100
    print(f"{prev['observation_date'].year}-{curr['observation_date'].year}: {growth:+.1f}% ({curr['value_numeric']-prev['value_numeric']:+.1f}pp)")

## 3. Usage Analysis

In [ ]:
# Mobile money and usage data
usage_indicators = ['ACC_MM_ACCOUNT', 'USG_TELEBIRR_USERS', 'USG_MPESA_USERS']
usage_data = observations[observations['indicator_code'].isin(usage_indicators)]

fig = go.Figure()
for indicator in usage_indicators:
    data = usage_data[usage_data['indicator_code'] == indicator]
    if len(data) > 0:
        fig.add_trace(go.Scatter(
            x=data['observation_date'],
            y=data['value_numeric'],
            mode='lines+markers',
            name=data['indicator'].iloc[0],
        ))

fig.update_layout(
    title='Digital Payment Usage Trends',
    xaxis_title='Year',
    yaxis_title='Value',
    template='plotly_white'
)
fig.show()

## 4. Infrastructure Analysis

In [ ]:
# Infrastructure trends
infra_indicators = ['ACC_MOBILE_PEN', 'ACC_INTERNET_PEN', 'AFF_GDP_PCAP']
infra_data = observations[observations['indicator_code'].isin(infra_indicators)]

fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for i, indicator in enumerate(infra_indicators):
    data = infra_data[infra_data['indicator_code'] == indicator].sort_values('observation_date')
    if len(data) > 0:
        axes[i].plot(data['observation_date'], data['value_numeric'], 'o-')
        axes[i].set_title(data['indicator'].iloc[0])
        axes[i].tick_params(axis='x', rotation=45)
        axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Event Timeline

In [ ]:
# Event timeline
events_sorted = events.sort_values('observation_date')

fig = go.Figure()
category_colors = {
    'product_launch': 'red',
    'market_entry': 'blue', 
    'policy': 'green',
    'infrastructure': 'orange'
}

for _, event in events_sorted.iterrows():
    color = category_colors.get(event['category'], 'gray')
    fig.add_trace(go.Scatter(
        x=[event['observation_date']],
        y=[event['category']],
        mode='markers',
        marker=dict(size=15, color=color),
        name=event['indicator'],
        text=f"{event['observation_date'].date()}<br>{event['indicator']}",
        hoverinfo='text'
    ))

fig.update_layout(
    title='Ethiopia Financial Inclusion Event Timeline',
    xaxis_title='Year',
    template='plotly_white',
    height=400
)
fig.show()

## 6. Correlation Analysis

In [ ]:
# Correlation matrix
pivot_data = observations.pivot_table(
    index='observation_date',
    columns='indicator_code', 
    values='value_numeric'
)

key_indicators = ['ACC_OWNERSHIP', 'ACC_MOBILE_PEN', 'ACC_INTERNET_PEN', 'AFF_GDP_PCAP']
available = [ind for ind in key_indicators if ind in pivot_data.columns]

if len(available) > 1:
    corr_matrix = pivot_data[available].corr()
    
    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, cmap='RdBu_r', center=0, fmt='.2f')
    plt.title('Correlation Matrix')
    plt.tight_layout()
    plt.show()
    
    print("Correlations with Account Ownership:")
    print(corr_matrix['ACC_OWNERSHIP'].sort_values(ascending=False))

## 7. Key Insights Summary

In [ ]:
print("=== KEY INSIGHTS ===")
print("\n1. Account Ownership Growth:")
print("   - Steady growth from 14% (2011) to 49% (2024)")
print("   - Notable slowdown: only +3pp from 2021-2024")
print("   - Despite 65M+ mobile money accounts registered")

print("\n2. Infrastructure Enablers:")
print("   - Mobile penetration: 38.5% → 74.5% (2015-2022)")
print("   - Internet penetration: 2.1% → 35.2% (2015-2022)")
print("   - Strong correlation with financial inclusion")

print("\n3. Event Impacts:")
print("   - Telebirr launch (2021): Major market catalyst")
print("   - M-Pesa entry (2022): Increased competition")
print("   - EthSwitch (2019): Infrastructure foundation")

print("\n4. Data Quality:")
print("   - High confidence: 96.2% of records")
print("   - Good temporal coverage: 2011-2025")
print("   - Missing: Gender-disaggregated data")